In [2]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, dates
from datetime import datetime, timedelta

# %run 'rsi_module.ipynb'


rc('font', family='D2Coding')
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(12, 6))

# api url
base_url = 'http://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo?serviceKey=9oUviJPMmodfrNLBPYD0czH%2BOoPnArWcnHsw6T5g2TwnKEs1dwXCp%2FI%2BNfeEGngIAK74J550ZjxJ0CxJy4cK7Q%3D%3D&resultType=json'

# API 호출 함수sd
def select_item(name, start, end):
    start = datetime.strptime(start, '%Y%m%d')
    ends = datetime.strptime(end, '%Y%m%d')
    # 시작일, 종료일 문자열을 dateformat으로 변경
    page = int((ends - start).days) + 20
    # 한 번에 가져올 데이터 계산 (종료일 - 시작일 + 20(이동 평균) )
    start = (start - timedelta(days=20)).strftime('%Y%m%d')
    # 이동 평균을 구하기 위해, 20일 전 데이터부터 가져오기
    url = f'{base_url}&numOfRows={page}&itmsNm={name}&beginBasDt={start}&endBasDt={end}'
    # API 호출 URL 생성
    # numOfRows: 한 번에 가져올 데이터의 개수
    # itemsNm: 종목명
    # beginBasDt: 시작일
    # endBasDt: 종료일
    # 만약 beginBasDt와 endBasDt를 이용시, {beginBasDt ~ endBasDt} 기간의 데이터를 가져옴
    return url;

# DataFrame 생성 함수
def make_df(url):
    # 데이터 전처리
    response = requests.get(url)
    if response.status_code == 200:
        contents = response.text
    else:
        print(f"API 호출 실패: {response.status_code}")
    json_ob = json.loads(contents)
    data = json_ob['response']['body']['items']['item']
    selected_data = []

    # 데이터 프레임 생성
    for item in data:
        selected_item = {
            'basDt': item['basDt'],
            'itmsNm': item['itmsNm'],
            'mrktCtg': item['mrktCtg'],
            'clpr': item['clpr'],
            'trqu': item['trqu'],
            # 'vs': '거래량' -> trqu로 변경
            # 'vs': 등락치
            'fltRt': item['fltRt'],
            'mkp': item['mkp'],
            'hipr': item['hipr'],
            'lopr': item['lopr']
        }
        selected_data.append(selected_item)
    dataframe = pd.DataFrame(selected_data)

    # 데이터 프레임 열 이름 변경
    dataframe = dataframe.rename(columns={
        'basDt': '날짜',
        'itmsNm': '종목명',
        'mrktCtg': '시장구분',
        'clpr': '종가',
        'trqu': '거래량',
        # 'vs': '거래량' -> trqu로 변경
        # 'vs': 등락치
        'fltRt': '등락률',
        'mkp': '시가총액',
        'hipr': '고가',
        'lopr': '저가'
    })

    # 날짜를 날짜 형식으로 변환
    dataframe['날짜'] = pd.to_datetime(dataframe['날짜'], format='%Y%m%d')

    # object를 수치로 변환
    dataframe['종가'] = pd.to_numeric(dataframe['종가'])
    dataframe['거래량'] = pd.to_numeric(dataframe['거래량'])
    dataframe['고가'] = pd.to_numeric(dataframe['고가'])
    dataframe['저가'] = pd.to_numeric(dataframe['저가'])
    dataframe['등락률'] = pd.to_numeric(dataframe['등락률'])
    dataframe['시가총액'] = pd.to_numeric(dataframe['시가총액'])

    # 날짜를 인덱스로 설정
    dataframe.set_index('날짜', inplace=True)

    # 일간 변동성 계산 (예: 20일 기간을 사용)
    period = 20
    dataframe['ma20'] = dataframe['종가'].rolling(window=period).mean()
    # 이동 평균 추가
    dataframe['일간변동성'] = dataframe['종가'].rolling(window=period).std()
    return dataframe

# 볼린저 밴드 계산 함수
def tuning_df_to_bollinger(df):
    df['upper'] = df['ma20'] + 2*df['일간변동성'] # 상단
    df['lower'] = df['ma20'] - 2*df['일간변동성'] # 하단
    df = df[19:]
    # 20일 기준으로 이동 평균을 구하기 때문에, 20일 이전 데이터는 제외
    return df

# 단일 기준(거래량, 종가, 등락률 등) 그래프
def draw_single_graph(df, filter, color):
    df = df[:19]
    # 이동 평균을 구하기 위해, 20일 이전 데이터를 가져왔기에 그를 제외한 데이터만을 보여주기 위해 사용
    plt.plot(df.index, df[filter], color = color)
    # 그래프 생성
    plt.scatter(df.index, df[filter], color = 'black')
    # 산점도 생성
    plt.xlabel('Date')
    plt.xticks(rotation=45)
    plt.ylabel(filter)
    plt.title(f'{filter} 기준 그래프')
    # 기타 정보
    ax = plt.gca()
    ax.xaxis.set_major_locator(dates.DayLocator())
    # x축 일 기준으로 간격 설정
    # DayLocator(days=n(default=1)): 일 간격
    plt.show()

# 지표 기준(볼린저, RSI 등) 그래프
def draw_graph(df, filter):
    if (filter == 'bollinger_band'):
        plt.plot(df.index, df['종가'], label='Close')
        plt.plot(df.index, df['upper'], linestyle='dashed', label='Upper band')
        plt.plot(df.index, df['ma20'], linestyle='dashed', label='Moving Average 20')
        plt.plot(df.index, df['lower'], linestyle='dashed', label='Lower band')
    elif (filter == 'rsi'):
        plt.plot(df.index, df['종가'], label = '종가')
    else:
        plt.plot(df.index, df['종가'], label = '종가')
    # 필터별로 그래프 생성
    plt.title(f'{filter} 기준 그래프')
    plt.xlabel('Date')
    plt.ylabel(filter)
    plt.xticks(rotation=45)
    ax = plt.gca()
    ax.xaxis.set_major_locator(dates.DayLocator())
    # 기타 정보
    plt.legend()

print(url)
url = select_item('삼성전자', '20231002', '20231105')
# API 호출
df.info()

df = make_df(url)
# DATAFRAME 생성

draw_single_graph(df, '거래량', 'red')
# 단일 기준 그래프

df = tuning_df_to_bollinger(df)
draw_graph(df, 'bollinger_band')
# 볼린저 밴드 기준 그래프

NameError: name 'url' is not defined

<Figure size 1200x600 with 0 Axes>